In [1]:
from google.colab import drive
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
import pickle
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Input, Flatten, Dense, Lambda, BatchNormalization, Dropout, GlobalAveragePooling2D, Concatenate, Activation, Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image


# Mount Google Drive
drive.mount('/content/drive')

# Load the labels
labels_path = '/content/drive/MyDrive/Contrastive Learning/contrastive_learning_labels.csv'
labels_df = pd.read_csv(labels_path)

# Directory path where images are stored
image_dir = '/content/drive/MyDrive/Contrastive Learning/contrastive_learning_images'

# Check if the image directory exists
if not os.path.exists(image_dir):
    print("Image directory not found!")
else:
    print("Images are ready for training.")

Mounted at /content/drive
Images are ready for training.


In [ ]:
labels_df['class'].value_counts()

,count
class,
1,366
2,183
0,168
3,63
4,48
5,29
6,17


# Creating Image Pairs (only run to create these pairs, dont run on GPU)

In [ ]:
def load_image(pothole_id, directory=image_dir, target_size=(224, 224)):
    img_path = os.path.join(directory, f'{pothole_id}.jpg')
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize the image
    return img

# Standardize the scalar features
def standardize_features(labels_df):
    scaler = StandardScaler()
    labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']] = scaler.fit_transform(
        labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']]
    )
    return labels_df, scaler

# Function to create pairs (positive and negative) with standardized scalar features and progress feedback
def create_pairs_with_features(labels_df, positive_pairs_per_image=5, negative_pairs_per_image=5):
    pairs = []
    scalar_features_1 = []
    scalar_features_2 = []
    labels = []

    grouped = labels_df.groupby('class')
    total_groups = len(grouped)

    # Calculate progress interval (at least 1)
    progress_interval = max(1, total_groups // 10)

    # Creating positive pairs (same class)
    for idx, (class_name, group) in enumerate(grouped):
        if idx % progress_interval == 0:
            print(f"Progress: {min(100, idx / total_groups * 100):.1f}% (Creating positive pairs)")

        pothole_ids = group['pothole_id'].values
        if len(pothole_ids) > 1:
            for i in range(len(pothole_ids)):
                img1 = load_image(pothole_ids[i])

                # Create multiple positive pairs for each image
                for j in range(positive_pairs_per_image):
                    img2_id = np.random.choice(pothole_ids)
                    while img2_id == pothole_ids[i]:
                        img2_id = np.random.choice(pothole_ids)  # Avoid pairing with itself

                    img2 = load_image(img2_id)
                    pairs.append((img1, img2))

                    # Collect corresponding scalar features
                    features_1 = group[group['pothole_id'] == pothole_ids[i]][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                    features_2 = group[group['pothole_id'] == img2_id][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                    scalar_features_1.append(features_1)
                    scalar_features_2.append(features_2)

                    labels.append(1)

    # Creating negative pairs (different classes)
    total_samples = len(labels_df)
    progress_interval = max(1, total_samples // 10)

    for i in range(total_samples):
        if i % progress_interval == 0:
            print(f"Progress: {min(100, i / total_samples * 100):.1f}% (Creating negative pairs)")

        class_names = labels_df['class'].unique()
        class1 = random.choice(class_names)
        class2 = random.choice([cls for cls in class_names if cls != class1])

        pothole_id_1 = labels_df[labels_df['class'] == class1].sample(1)['pothole_id'].values[0]
        img1 = load_image(pothole_id_1)

        # Create multiple negative pairs for each image
        for j in range(negative_pairs_per_image):
            pothole_id_2 = labels_df[labels_df['class'] == class2].sample(1)['pothole_id'].values[0]
            img2 = load_image(pothole_id_2)

            pairs.append((img1, img2))

            # Collect corresponding scalar features
            features_1 = labels_df[labels_df['pothole_id'] == pothole_id_1][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
            features_2 = labels_df[labels_df['pothole_id'] == pothole_id_2][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
            scalar_features_1.append(features_1)
            scalar_features_2.append(features_2)

            labels.append(0)

    print("Progress: 100% (Pair creation completed)")
    return np.array(pairs), np.array(scalar_features_1), np.array(scalar_features_2), np.array(labels)

labels_df, scaler = standardize_features(labels_df)
# Example usage
image_pairs, scalar_features_1, scalar_features_2, pair_labels = create_pairs_with_features(labels_df, positive_pairs_per_image=5, negative_pairs_per_image=5)

Progress: 0.0% (Creating positive pairs)
Progress: 14.3% (Creating positive pairs)
Progress: 28.6% (Creating positive pairs)
Progress: 42.9% (Creating positive pairs)
Progress: 57.1% (Creating positive pairs)
Progress: 71.4% (Creating positive pairs)
Progress: 85.7% (Creating positive pairs)
Progress: 0.0% (Creating negative pairs)
Progress: 10.0% (Creating negative pairs)
Progress: 19.9% (Creating negative pairs)
Progress: 29.9% (Creating negative pairs)
Progress: 39.8% (Creating negative pairs)
Progress: 49.8% (Creating negative pairs)
Progress: 59.7% (Creating negative pairs)
Progress: 69.7% (Creating negative pairs)
Progress: 79.6% (Creating negative pairs)
Progress: 89.6% (Creating negative pairs)
Progress: 99.5% (Creating negative pairs)
Progress: 100% (Pair creation completed)


# Saving in batches

In [ ]:
def load_image(pothole_id, directory=image_dir, target_size=(224, 224)):
    img_path = os.path.join(directory, f'{pothole_id}.jpg')
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize the image
    return img

# Standardize the scalar features
def standardize_features(labels_df):
    scaler = StandardScaler()
    labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']] = scaler.fit_transform(
        labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']]
    )
    return labels_df, scaler

def create_and_save_pairs(labels_df, batch_size=1000, positive_pairs_per_image=2, negative_pairs_per_image=2, save_dir='/content/drive/MyDrive/pairs_batches'):
    os.makedirs(save_dir, exist_ok=True)
    batch_count = 0

    grouped = labels_df.groupby('class')
    total_groups = len(grouped)

    # Calculate progress interval (at least 1)
    progress_interval = max(1, total_groups // 10)

    def save_batch(batch_id, pairs, scalar_features_1, scalar_features_2, labels):
        np.save(os.path.join(save_dir, f'pairs_batch_{batch_id}.npy'), pairs)
        np.save(os.path.join(save_dir, f'scalar_features_1_batch_{batch_id}.npy'), scalar_features_1)
        np.save(os.path.join(save_dir, f'scalar_features_2_batch_{batch_id}.npy'), scalar_features_2)
        np.save(os.path.join(save_dir, f'labels_batch_{batch_id}.npy'), labels)

    pairs = []
    scalar_features_1 = []
    scalar_features_2 = []
    labels = []

    # Creating positive and negative pairs in batches
    for idx, (class_name, group) in enumerate(grouped):
        if idx % progress_interval == 0:
            print(f"Progress: {min(100, idx / total_groups * 100):.1f}% (Processing pairs)")

        pothole_ids = group['pothole_id'].values
        if len(pothole_ids) > 1:
            for i in range(len(pothole_ids)):
                img1 = load_image(pothole_ids[i])

                # Create multiple positive pairs for each image
                for j in range(positive_pairs_per_image):
                    img2_id = np.random.choice(pothole_ids)
                    while img2_id == pothole_ids[i]:
                        img2_id = np.random.choice(pothole_ids)  # Avoid pairing with itself

                    img2 = load_image(img2_id)
                    pairs.append((img1, img2))

                    # Collect corresponding scalar features
                    features_1 = group[group['pothole_id'] == pothole_ids[i]][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                    features_2 = group[group['pothole_id'] == img2_id][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                    scalar_features_1.append(features_1)
                    scalar_features_2.append(features_2)

                    labels.append(1)

                # Create multiple negative pairs for each image
                for j in range(negative_pairs_per_image):
                    class_names = labels_df['class'].unique()
                    class2 = np.random.choice([cls for cls in class_names if cls != class_name])

                    pothole_id_2 = labels_df[labels_df['class'] == class2].sample(1)['pothole_id'].values[0]
                    img2 = load_image(pothole_id_2)

                    pairs.append((img1, img2))

                    # Collect corresponding scalar features
                    features_1 = group[group['pothole_id'] == pothole_ids[i]][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                    features_2 = labels_df[labels_df['pothole_id'] == pothole_id_2][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                    scalar_features_1.append(features_1)
                    scalar_features_2.append(features_2)

                    labels.append(0)

                # Check if we reached the batch size limit
                if len(pairs) >= batch_size:
                    save_batch(batch_count, pairs, scalar_features_1, scalar_features_2, labels)
                    pairs, scalar_features_1, scalar_features_2, labels = [], [], [], []  # Reset the lists
                    batch_count += 1

    # Save the remaining pairs
    if len(pairs) > 0:
        save_batch(batch_count, pairs, scalar_features_1, scalar_features_2, labels)

    print(f"Progress: 100% (Pair creation completed in {batch_count + 1} batches)")

# Example usage
labels_df, scaler = standardize_features(labels_df)
create_and_save_pairs(labels_df, batch_size=1000, positive_pairs_per_image=5, negative_pairs_per_image=5)

Progress: 0.0% (Processing pairs)
Progress: 14.3% (Processing pairs)
Progress: 28.6% (Processing pairs)
Progress: 42.9% (Processing pairs)
Progress: 57.1% (Processing pairs)
Progress: 71.4% (Processing pairs)
Progress: 85.7% (Processing pairs)
Progress: 100% (Pair creation completed in 9 batches)


## Saving Image Pairs

In [ ]:
with open('/content/drive/MyDrive/Contrastive Learning/contrastive_pairs_with_features.pkl', 'wb') as f:
    pickle.dump((image_pairs, scalar_features_1, scalar_features_2, pair_labels), f)

## Loading Image Pairs

In [ ]:
with open('/content/drive/MyDrive/Contrastive Learning/contrastive_pairs_with_features.pkl', 'rb') as f:
    image_pairs, scalar_features_1, scalar_features_2, pair_labels = pickle.load(f)

# Modelling

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dense, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def load_image(pothole_id, directory, target_size=(224, 224)):
    img_path = os.path.join(directory, f'{pothole_id}.jpg')
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize the image
    return img

def data_generator(save_dir, batch_size=32):
    pairs_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('pairs_batch_') and f.endswith('.npy')])
    scalar_features_1_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('scalar_features_1_batch_') and f.endswith('.npy')])
    scalar_features_2_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('scalar_features_2_batch_') and f.endswith('.npy')])
    labels_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('labels_batch_') and f.endswith('.npy')])

    while True:
        for pair_file, sf1_file, sf2_file, label_file in zip(pairs_batches, scalar_features_1_batches, scalar_features_2_batches, labels_batches):
            pairs = np.load(os.path.join(save_dir, pair_file))
            scalar_features_1 = np.load(os.path.join(save_dir, sf1_file))
            scalar_features_2 = np.load(os.path.join(save_dir, sf2_file))
            labels = np.load(os.path.join(save_dir, label_file))

            for i in range(0, len(pairs), batch_size):
                batch_pairs = pairs[i:i + batch_size]
                batch_sf1 = scalar_features_1[i:i + batch_size]
                batch_sf2 = scalar_features_2[i:i + batch_size]
                batch_labels = labels[i:i + batch_size]

                # Convert numpy arrays to TensorFlow tensors
                yield (
                    (
                        tf.convert_to_tensor(batch_pairs[:, 0], dtype=tf.float32),
                        tf.convert_to_tensor(batch_sf1, dtype=tf.float32),
                        tf.convert_to_tensor(batch_pairs[:, 1], dtype=tf.float32),
                        tf.convert_to_tensor(batch_sf2, dtype=tf.float32)
                    ),
                    tf.convert_to_tensor(batch_labels, dtype=tf.float32)
                )

In [ ]:
def make_embedding_model_with_scalars(input_shape=(224, 224, 3), scalar_shape=(2,), embedding_dim=128):
    img_input = Input(shape=input_shape, name='input_image')

    # Convolutional layers
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same', name='c1')(img_input)
    b1 = BatchNormalization()(c1)
    m1 = MaxPooling2D((2, 2), name='m1')(b1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same', name='c2')(m1)
    b2 = BatchNormalization()(c2)
    m2 = MaxPooling2D((2, 2), name='m2')(b2)

    c3 = Conv2D(128, (3, 3), activation='relu', padding='same', name='c3')(m2)
    b3 = BatchNormalization()(c3)
    m3 = MaxPooling2D((2, 2), name='m3')(b3)

    c4 = Conv2D(128, (3, 3), activation='relu', padding='same', name='c4')(m3)
    b4 = BatchNormalization()(c4)
    m4 = MaxPooling2D((2, 2), name='m4')(b4)

    f1 = GlobalAveragePooling2D(name='flatten')(m4)

    scalar_input = Input(shape=scalar_shape, name='input_scalar')
    combined = Concatenate(name='concatenate_embedding_scalars')([f1, scalar_input])

    embedding = Dense(embedding_dim, activation='relu', name='embedding')(combined)
    embedding = BatchNormalization()(embedding)

    return Model([img_input, scalar_input], embedding, name='embedding_model')

In [ ]:
def create_contrastive_model(input_shape=(224, 224, 3), scalar_shape=(2,), embedding_dim=128):
    embedding_model = make_embedding_model_with_scalars(input_shape=input_shape, scalar_shape=scalar_shape, embedding_dim=embedding_dim)

    input_a = Input(shape=input_shape, name='input_img_a')
    input_b = Input(shape=input_shape, name='input_img_b')

    scalar_input_a = Input(shape=scalar_shape, name='input_scalar_a')
    scalar_input_b = Input(shape=scalar_shape, name='input_scalar_b')

    embedding_a = embedding_model([input_a, scalar_input_a])
    embedding_b = embedding_model([input_b, scalar_input_b])

    distance = Lambda(lambda embeddings: tf.sqrt(tf.reduce_sum(tf.square(embeddings[0] - embeddings[1]), axis=-1)),
                      name='distance')([embedding_a, embedding_b])

    contrastive_model = Model(inputs=[input_a, scalar_input_a, input_b, scalar_input_b], outputs=distance)

    return contrastive_model

In [ ]:
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def call(self, y_true, y_pred):
        positive_pairs = y_true * tf.square(y_pred)
        negative_pairs = (1 - y_true) * tf.square(tf.maximum(self.margin - y_pred, 0))
        return tf.reduce_mean(positive_pairs + negative_pairs)

contrastive_model = create_contrastive_model()

contrastive_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                          loss=ContrastiveLoss(margin=1.0))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Contrastive Learning/best_contrastive_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(save_dir='/content/drive/MyDrive/pairs_batches', batch_size=32),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),  # Input image A
            tf.TensorSpec(shape=(None, 2), dtype=tf.float32),            # Scalar features A
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),  # Input image B
            tf.TensorSpec(shape=(None, 2), dtype=tf.float32)             # Scalar features B
        ),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)                  # Labels
    )
).prefetch(tf.data.AUTOTUNE)

# Compile and train the model
history = contrastive_model.fit(
    train_dataset,
    epochs=50,
    steps_per_epoch=100,  # Adjust this based on your dataset size
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

Epoch 1/50
  4/100 ━━━━━━━━━━━━━━━━━━━━ 18:57 12s/step - loss: 102.5786

KeyboardInterrupt: 

In [ ]:
def make_embedding_model_with_scalars(input_shape=(224, 224, 3), scalar_shape=(2,), embedding_dim=128):
    # Image input branch
    img_input = Input(shape=input_shape, name='input_image')

    # Convolutional layers
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same', name='c1')(img_input)
    b1 = BatchNormalization()(c1)
    m1 = MaxPooling2D((2, 2), name='m1')(b1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same', name='c2')(m1)
    b2 = BatchNormalization()(c2)
    m2 = MaxPooling2D((2, 2), name='m2')(b2)

    c3 = Conv2D(128, (3, 3), activation='relu', padding='same', name='c3')(m2)
    b3 = BatchNormalization()(c3)
    m3 = MaxPooling2D((2, 2), name='m3')(b3)

    c4 = Conv2D(128, (3, 3), activation='relu', padding='same', name='c4')(m3)
    b4 = BatchNormalization()(c4)
    m4 = MaxPooling2D((2, 2), name='m4')(b4)

    # Global average pooling and dense layer for embedding
    f1 = GlobalAveragePooling2D(name='flatten')(m4)

    # Scalar input branch
    scalar_input = Input(shape=scalar_shape, name='input_scalar')

    # Combine image embedding and scalar features
    combined = Concatenate(name='concatenate_embedding_scalars')([f1, scalar_input])

    # Final embedding layer
    embedding = Dense(embedding_dim, activation='relu', name='embedding')(combined)
    embedding = BatchNormalization()(embedding)

    return Model([img_input, scalar_input], embedding, name='embedding_model')

# Define the input shapes
image_shape = (224, 224, 3)
scalar_shape = (2,)

# Instantiate the embedding model with scalar features
embedding_model = make_embedding_model_with_scalars(input_shape=image_shape, scalar_shape=scalar_shape, embedding_dim=128)

# Define the inputs for the contrastive model
input_a = Input(shape=image_shape, name='input_img_a')
input_b = Input(shape=image_shape, name='input_img_b')

scalar_input_a = Input(shape=scalar_shape, name='input_scalar_a')
scalar_input_b = Input(shape=scalar_shape, name='input_scalar_b')

# Generate embeddings for both inputs (image + scalar features)
embedding_a = embedding_model([input_a, scalar_input_a])
embedding_b = embedding_model([input_b, scalar_input_b])

# Compute the L2 distance between the embeddings
distance = Lambda(lambda embeddings: tf.sqrt(tf.reduce_sum(tf.square(embeddings[0] - embeddings[1]), axis=-1)),
                  name='distance')([embedding_a, embedding_b])

# Define the contrastive model
contrastive_model = Model(inputs=[input_a, scalar_input_a, input_b, scalar_input_b], outputs=distance)

class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def call(self, y_true, y_pred):
        positive_pairs = y_true * tf.square(y_pred)
        negative_pairs = (1 - y_true) * tf.square(tf.maximum(self.margin - y_pred, 0))
        return tf.reduce_mean(positive_pairs + negative_pairs)

# Compile the model with contrastive loss
contrastive_model.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss=ContrastiveLoss(margin=1.0))

# Summary of the contrastive model
contrastive_model.summary()

# Now, you can train the model with your image pairs and scalar features
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Contrastive Learning/best_contrastive_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model with early stopping and checkpoints
history = contrastive_model.fit(
    [image_pairs[:, 0], scalar_features_1, image_pairs[:, 1], scalar_features_2],
    pair_labels,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping, model_checkpoint]
)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_img_a (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_scalar_a            │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_img_b (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_scalar_b            │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_model           │ (None, 128)            │        259,520 │ input_img_a[0][0],     │
│ (Functional)              │                        │                │ input_scalar_a[0][0],  │
│                           │                        │                │ input_img_b[0][0],     │
│                           │                        │                │ input_scalar_b[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ distance (Lambda)         │ (None)                 │              0 │ embedding_model[0][0], │
│                           │                        │                │ embedding_model[1][0]  │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 259,520 (1013.75 KB)

 Trainable params: 258,560 (1010.00 KB)

 Non-trainable params: 960 (3.75 KB)

Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 570s 13s/step - loss: 111.1808 - val_loss: 0.0158
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 618s 13s/step - loss: 72.5914 - val_loss: 0.0014
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 555s 13s/step - loss: 41.7505 - val_loss: 0.0000e+00
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 559s 12s/step - loss: 34.4217 - val_loss: 0.0000e+00
Epoch 5/50
19/44 ━━━━━━━━━━━━━━━━━━━━ 5:02 12s/step - loss: 16.8252

In [ ]:
embedding_model.save('/content/drive/MyDrive/Contrastive Learning/embedding_model.keras')

# Triplett Loss Contrastive Learning

In [ ]:
def load_image(pothole_id, directory=image_dir, target_size=(224, 224)):
    img_path = os.path.join(directory, f'{pothole_id}.jpg')
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize the image
    return img

# Standardize the scalar features
def standardize_features(labels_df):
    scaler = StandardScaler()
    labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']] = scaler.fit_transform(
        labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']]
    )
    return labels_df, scaler

# Function to create triplets with standardized scalar features
def create_triplets(labels_df):
    triplets = []
    scalar_features_anchor = []
    scalar_features_positive = []
    scalar_features_negative = []

    grouped = labels_df.groupby('class')
    total_groups = len(grouped)

    progress_interval = max(1, total_groups // 10)

    for idx, (class_name, group) in enumerate(grouped):
        if idx % progress_interval == 0:
            print(f"Progress: {min(100, idx / total_groups * 100):.1f}% (Creating triplets)")

        pothole_ids = group['pothole_id'].values
        if len(pothole_ids) > 1:
            for i in range(len(pothole_ids) - 1):
                anchor_img = load_image(pothole_ids[i])
                positive_img = load_image(pothole_ids[i + 1])

                negative_class = random.choice([cls for cls in labels_df['class'].unique() if cls != class_name])
                negative_img_id = labels_df[labels_df['class'] == negative_class].sample(1)['pothole_id'].values[0]
                negative_img = load_image(negative_img_id)

                triplets.append((anchor_img, positive_img, negative_img))

                # Collect corresponding scalar features
                anchor_features = group[group['pothole_id'] == pothole_ids[i]][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                positive_features = group[group['pothole_id'] == pothole_ids[i + 1]][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                negative_features = labels_df[labels_df['pothole_id'] == negative_img_id][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]

                scalar_features_anchor.append(anchor_features)
                scalar_features_positive.append(positive_features)
                scalar_features_negative.append(negative_features)

    print("Progress: 100% (Triplet creation completed)")
    return np.array(triplets), np.array(scalar_features_anchor), np.array(scalar_features_positive), np.array(scalar_features_negative)

# Standardize the scalar features in the labels dataframe
labels_df, scaler = standardize_features(labels_df)

# Create the triplets and corresponding scalar feature pairs
triplets, scalar_features_anchor, scalar_features_positive, scalar_features_negative = create_triplets(labels_df)

## Saving image pairs

In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/Contrastive Learning/triplets_and_features.pkl', 'wb') as f:
    pickle.dump((triplets, scalar_features_anchor, scalar_features_positive, scalar_features_negative), f)

## Loading image pairs

In [ ]:
with open('/content/drive/MyDrive/Contrastive Learning/triplets_and_features.pkl', 'rb') as f:
    triplets, scalar_features_anchor, scalar_features_positive, scalar_features_negative = pickle.load(f)

# Triplets with Hard Negatives

In [ ]:
def load_image(pothole_id, directory=image_dir, target_size=(224, 224)):
    img_path = os.path.join(directory, f'{pothole_id}.jpg')
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize the image
    return img

# Standardize the scalar features
def standardize_features(labels_df):
    scaler = StandardScaler()
    labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']] = scaler.fit_transform(
        labels_df[['pothole_area_mm2', 'mm_to_pixel_ratio']]
    )
    return labels_df, scaler

def create_triplets_with_hard_negatives(labels_df):
    triplets = []
    scalar_features_anchor = []
    scalar_features_positive = []
    scalar_features_negative = []

    grouped = labels_df.groupby('class')
    total_groups = len(grouped)

    progress_interval = max(1, total_groups // 10)

    for idx, (class_name, group) in enumerate(grouped):
        if idx % progress_interval == 0:
            print(f"Progress: {min(100, idx / total_groups * 100):.1f}% (Creating triplets with hard negatives)")

        pothole_ids = group['pothole_id'].values
        if len(pothole_ids) > 1:
            for i in range(len(pothole_ids) - 1):
                anchor_img = load_image(pothole_ids[i])
                positive_img = load_image(pothole_ids[i + 1])

                # Hard negative mining: find the hardest negative (most similar to the anchor)
                hardest_negative_distance = float('inf')
                hardest_negative_img_id = None

                for other_class in labels_df['class'].unique():
                    if other_class != class_name:
                        negative_img_id = labels_df[labels_df['class'] == other_class].sample(1)['pothole_id'].values[0]
                        negative_img = load_image(negative_img_id)

                        # Compute the distance in feature space
                        neg_distance = np.sum(np.square(anchor_img - negative_img))
                        if neg_distance < hardest_negative_distance:
                            hardest_negative_distance = neg_distance
                            hardest_negative_img_id = negative_img_id

                negative_img = load_image(hardest_negative_img_id)

                triplets.append((anchor_img, positive_img, negative_img))

                # Collect corresponding scalar features
                anchor_features = group[group['pothole_id'] == pothole_ids[i]][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                positive_features = group[group['pothole_id'] == pothole_ids[i + 1]][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]
                negative_features = labels_df[labels_df['pothole_id'] == hardest_negative_img_id][['pothole_area_mm2', 'mm_to_pixel_ratio']].values[0]

                scalar_features_anchor.append(anchor_features)
                scalar_features_positive.append(positive_features)
                scalar_features_negative.append(negative_features)

    print("Progress: 100% (Hard negative triplet creation completed)")
    return np.array(triplets), np.array(scalar_features_anchor), np.array(scalar_features_positive), np.array(scalar_features_negative)

# Create the triplets and corresponding scalar feature pairs with hard negative mining
labels_df, scaler = standardize_features(labels_df)

triplets, scalar_features_anchor, scalar_features_positive, scalar_features_negative = create_triplets_with_hard_negatives(labels_df)

Progress: 0.0% (Creating triplets with hard negatives)
Progress: 14.3% (Creating triplets with hard negatives)
Progress: 28.6% (Creating triplets with hard negatives)
Progress: 42.9% (Creating triplets with hard negatives)
Progress: 57.1% (Creating triplets with hard negatives)
Progress: 71.4% (Creating triplets with hard negatives)
Progress: 85.7% (Creating triplets with hard negatives)
Progress: 100% (Hard negative triplet creation completed)


## Save hard negative triplets

In [ ]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/Contrastive Learning/triplets_with_features_hard_negs.pkl', 'wb') as f:
    pickle.dump((triplets, scalar_features_anchor, scalar_features_positive, scalar_features_negative), f)

## Load hard negative triplets

In [ ]:
with open('/content/drive/MyDrive/Contrastive Learning/triplets_with_features_hard_negs.pkl', 'rb') as f:
    triplets, scalar_features_anchor, scalar_features_positive, scalar_features_negative = pickle.load(f)

# Model

In [ ]:
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import callbacks
import numpy as np

def create_simple_cnn_model_with_batchnorm(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
    ])
    return model

# Function to create the triplet model with scalar features included
def create_complex_triplet_model_with_features(image_shape, scalar_shape):
    # Load EfficientNetB0 with pretrained ImageNet weights, exclude top layers
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=image_shape)
    base_model.trainable = False  # Freeze the base model

    # Add custom layers on top of EfficientNetB0
    x = layers.GlobalAveragePooling2D()(base_model.output)  # Reduce to 1D vector
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Inputs
    input_1 = layers.Input(shape=image_shape)
    input_2 = layers.Input(shape=image_shape)
    input_3 = layers.Input(shape=image_shape)

    # Pass all three inputs through the EfficientNetB0 base model
    encoded_1 = base_model(input_1)
    encoded_2 = base_model(input_2)
    encoded_3 = base_model(input_3)

    encoded_1 = layers.GlobalAveragePooling2D()(encoded_1)
    encoded_2 = layers.GlobalAveragePooling2D()(encoded_2)
    encoded_3 = layers.GlobalAveragePooling2D()(encoded_3)

    encoded_1 = layers.Dense(512, activation='relu')(encoded_1)
    encoded_2 = layers.Dense(512, activation='relu')(encoded_2)
    encoded_3 = layers.Dense(512, activation='relu')(encoded_3)

    encoded_1 = layers.Dropout(0.5)(encoded_1)
    encoded_2 = layers.Dropout(0.5)(encoded_2)
    encoded_3 = layers.Dropout(0.5)(encoded_3)

    # Scalar features input
    scalar_input_1 = layers.Input(shape=scalar_shape)
    scalar_input_2 = layers.Input(shape=scalar_shape)
    scalar_input_3 = layers.Input(shape=scalar_shape)

    # Concatenate encoded images with scalar features
    merged_1 = layers.Concatenate()([encoded_1, scalar_input_1])
    merged_2 = layers.Concatenate()([encoded_2, scalar_input_2])
    merged_3 = layers.Concatenate()([encoded_3, scalar_input_3])

    # Merge all three to create embeddings
    triplet_output = layers.Concatenate()([merged_1, merged_2, merged_3])

    model = models.Model(inputs=[input_1, input_2, input_3, scalar_input_1, scalar_input_2, scalar_input_3], outputs=triplet_output)
    return model

# Triplet loss function
def triplet_loss(margin=1):
    def loss(y_true, y_pred):
        anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)
        positive_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        negative_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        return tf.reduce_mean(tf.maximum(positive_distance - negative_distance + margin, 0.0))
    return loss

# Compile the model with triplet loss
image_shape = (224, 224, 3)
scalar_shape = (2,)  # Assuming you have 2 scalar features (mm_to_pixel_ratio, pothole_area)
triplet_model = create_complex_triplet_model_with_features(image_shape, scalar_shape)

triplet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=triplet_loss(margin=1))

triplet_model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb0            │ (None, 7, 7, 1280)     │      4,049,571 │ input_layer_1[0][0],   │
│ (Functional)              │                        │                │ input_layer_2[0][0],   │
│                           │                        │                │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[0][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[1][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 1280)           │              0 │ efficientnetb0[2][0]   │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 512)            │        655,872 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 512)            │        655,872 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 512)            │        655,872 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 512)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 512)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)  

 Total params: 6,017,187 (22.95 MB)

 Trainable params: 1,967,616 (7.51 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=50,
                               restore_best_weights=True,
                               verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              min_lr=1e-7,
                              verbose=1)

model_checkpoint = callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/Contrastive Learning/best_triplet_model.keras',
                                             monitor='val_loss',
                                             save_best_only=True,
                                             verbose=1)

# Prepare triplet inputs and train the model
triplet_model.fit(
    [triplets[:, 0], triplets[:, 1], triplets[:, 2], scalar_features_anchor, scalar_features_positive, scalar_features_negative],
    np.zeros(len(triplets)),  # Dummy labels; triplet loss doesn't need true labels
    validation_split=0.2,
    epochs=500,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1
)

Epoch 1/500
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.7064   
Epoch 1: val_loss improved from inf to 3.09075, saving model to /content/drive/MyDrive/Contrastive Learning/best_triplet_model.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 120s 3s/step - loss: 0.6929 - val_loss: 3.0908 - learning_rate: 1.0000e-04
Epoch 2/500
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.0823
Epoch 2: val_loss improved from 3.09075 to 2.69527, saving model to /content/drive/MyDrive/Contrastive Learning/best_triplet_model.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 54s 188ms/step - loss: 0.0824 - val_loss: 2.6953 - learning_rate: 1.0000e-04
Epoch 3/500
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.0826
Epoch 3: val_loss improved from 2.69527 to 2.52356, saving model to /content/drive/MyDrive/Contrastive Learning/best_triplet_model.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 209ms/step - loss: 0.0815 - val_loss: 2.5236 - learning_rate: 1.0000e-04
Epoch 4/500
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.0456
Epoch 4: val

## Saving Model and Weights

In [ ]:
from tensorflow.keras.models import Model

# Recreate the embedding model with both image and scalar inputs
embedding_model = Model(
    inputs=triplet_model.input,
    outputs=triplet_model.get_layer('concatenate_3').output  # Use the final concatenation layer as output
)

# Save the embedding model properly
embedding_model.save('/content/drive/MyDrive/Contrastive Learning/embedding_model_with_scalars.keras')

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0


# Modelling ONly Using Pixel Ratio

In [ ]:
def residual_block(x, filters, kernel_size=3, stride=1, activation='relu', dropout_rate=0.4):
    res = Conv2D(filters, kernel_size, strides=stride, padding='same', activation=activation)(x)
    res = BatchNormalization()(res)
    res = Dropout(dropout_rate)(res)

    res = Conv2D(filters, kernel_size, strides=1, padding='same', activation=None)(res)
    res = BatchNormalization()(res)

    if x.shape[-1] != filters:
        x = Conv2D(filters, kernel_size=(1, 1), strides=stride, padding='same', activation=None)(x)

    res = Add()([x, res])
    res = Activation(activation)(res)
    return res

def make_embedding_model(input_shape=(224, 224, 3), scalar_shape=(1,), embedding_dim=256, dropout_rate=0.4):
    inp = Input(shape=input_shape, name='input_image')

    c1 = Conv2D(64, (7, 7), strides=2, activation='relu', padding='same', name='conv_layer_1')(inp)
    c1 = BatchNormalization()(c1)
    m1 = MaxPooling2D((3, 3), strides=2, padding='same', name='pool_layer_1')(c1)

    r1 = residual_block(m1, 128, dropout_rate=dropout_rate)
    r2 = residual_block(r1, 256, dropout_rate=dropout_rate)
    r3 = residual_block(r2, 512, dropout_rate=dropout_rate)  # Adding an additional residual block
    r4 = residual_block(r3, 512, dropout_rate=dropout_rate)  # Adding another additional residual block

    f1 = GlobalAveragePooling2D(name='global_avg_pool')(r4)  # Updated to use the final residual block

    scalar_input = Input(shape=scalar_shape, name='input_scalar')
    scalar_dense = Dense(128, activation='relu')(scalar_input)  # Increased the number of units
    scalar_dense = BatchNormalization()(scalar_dense)
    scalar_dense = Dropout(dropout_rate)(scalar_dense)

    combined = Concatenate(name='concat_image_scalar')([f1, scalar_dense])

    dense_1 = Dense(1024, activation='relu', name='dense_layer_1')(combined)  # Increased the number of units
    dense_1 = BatchNormalization()(dense_1)
    dense_1 = Dropout(dropout_rate)(dense_1)

    dense_2 = Dense(512, activation='relu', name='dense_layer_2')(dense_1)  # Added another dense layer
    dense_2 = BatchNormalization()(dense_2)
    dense_2 = Dropout(dropout_rate)(dense_2)

    embedding = Dense(embedding_dim, name='embedding_layer')(dense_2)
    embedding = BatchNormalization(name='embedding_batch_norm')(embedding)

    return Model(inputs=[inp, scalar_input], outputs=embedding, name='embedding_model')

class RandomSaturation(tf.keras.layers.Layer):
    def __init__(self, factor=0.2, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor

    def call(self, images, training=None):
        if training:
            return tf.image.random_saturation(images, 1 - self.factor, 1 + self.factor)
        return images

class RandomHue(tf.keras.layers.Layer):
    def __init__(self, factor=0.2, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor

    def call(self, images, training=None):
        if training:
            return tf.image.random_hue(images, self.factor)
        return images

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomContrast(factor=0.2),
    tf.keras.layers.RandomBrightness(factor=0.2),
    RandomSaturation(factor=0.2),  # Custom saturation layer
    RandomHue(factor=0.2),         # Custom hue layer
    # Custom layer to add Gaussian noise or Random Cutout if desired
])

# Define the learning rate scheduler
def lr_scheduler(epoch, lr):
    max_lr = 1e-3
    min_lr = 1e-6
    total_epochs = 500
    return min_lr + 0.5 * (max_lr - min_lr) * (1 + np.cos(np.pi * epoch / total_epochs))

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

# Adjust the data generator to include augmentation
def data_generator(save_dir, batch_size=32, augment=False):
    pairs_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('pairs_batch_') and f.endswith('.npy')])
    scalar_features_1_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('scalar_features_1_batch_') and f.endswith('.npy')])
    scalar_features_2_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('scalar_features_2_batch_') and f.endswith('.npy')])
    labels_batches = sorted([f for f in os.listdir(save_dir) if f.startswith('labels_batch_') and f.endswith('.npy')])

    while True:
        # Shuffle the batches together
        combined_batches = list(zip(pairs_batches, scalar_features_1_batches, scalar_features_2_batches, labels_batches))
        np.random.shuffle(combined_batches)

        for pair_file, sf1_file, sf2_file, label_file in combined_batches:
            pairs = np.load(os.path.join(save_dir, pair_file))
            scalar_features_1 = np.load(os.path.join(save_dir, sf1_file))[:, 1:2]  # Only take the pixel_to_mm_ratio
            scalar_features_2 = np.load(os.path.join(save_dir, sf2_file))[:, 1:2]  # Only take the pixel_to_mm_ratio
            labels = np.load(os.path.join(save_dir, label_file))

            num_batches = len(pairs) // batch_size
            for i in range(num_batches):
                batch_pairs = pairs[i * batch_size:(i + 1) * batch_size]
                batch_sf1 = scalar_features_1[i * batch_size:(i + 1) * batch_size]
                batch_sf2 = scalar_features_2[i * batch_size:(i + 1) * batch_size]
                batch_labels = labels[i * batch_size:(i + 1) * batch_size]

                if augment:
                    batch_pairs[:, 0] = data_augmentation(batch_pairs[:, 0])
                    batch_pairs[:, 1] = data_augmentation(batch_pairs[:, 1])

                yield (
                    (
                        tf.convert_to_tensor(batch_pairs[:, 0], dtype=tf.float32),
                        tf.convert_to_tensor(batch_sf1, dtype=tf.float32),
                        tf.convert_to_tensor(batch_pairs[:, 1], dtype=tf.float32),
                        tf.convert_to_tensor(batch_sf2, dtype=tf.float32)
                    ),
                    tf.convert_to_tensor(batch_labels, dtype=tf.float32)
                )

# Define the input shapes
image_shape = (224, 224, 3)
scalar_shape = (1,)

# Load the embedding model with scalar features included
base_network = make_embedding_model(input_shape=image_shape, scalar_shape=scalar_shape, embedding_dim=128)

# Input tensors for the two images and scalar features
input_a = Input(shape=image_shape, name='input_img_a')
input_b = Input(shape=image_shape, name='input_img_b')

scalar_input_a = Input(shape=scalar_shape, name='input_scalar_a')
scalar_input_b = Input(shape=scalar_shape, name='input_scalar_b')

# Generate embeddings for both inputs (image + scalar features)
embedding_a = base_network([input_a, scalar_input_a])
embedding_b = base_network([input_b, scalar_input_b])

# Calculate the distance between the embeddings
distance_layer = Lambda(lambda x: tf.math.square(x[0] - x[1]), name='distance_layer')
distance_output = distance_layer([embedding_a, embedding_b])

# Add classification layer
classifier = Dense(1, activation='sigmoid', name='classifier')(distance_output)

# Define the contrastive model with scalar inputs
model = Model(inputs=[input_a, scalar_input_a, input_b, scalar_input_b], outputs=[classifier, distance_output])

# Define the contrastive loss function
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def call(self, y_true, y_pred):
        label = tf.cast(y_true, tf.float32)
        neg_dist = tf.maximum(self.margin - y_pred, 0)
        return tf.reduce_mean(label * y_pred + (1.0 - label) * neg_dist, axis=-1)

# Instantiate loss functions
loss_contrastive = ContrastiveLoss(margin=1.0)
loss_classifier = tf.keras.losses.BinaryCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)  # Set a constant learning rate

# Compile the model
model.compile(
    loss=[loss_classifier, loss_contrastive],
    optimizer=optimizer,
    loss_weights=[1.0, 1.0],
    metrics=[['accuracy'], []]  # accuracy for classifier, no metrics for the contrastive loss
)

# Training dataset (no validation dataset)
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(save_dir='/content/drive/MyDrive/pairs_batches', batch_size=32, augment=True),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),  # Input image A
            tf.TensorSpec(shape=(None, 1), dtype=tf.float32),            # Scalar features A (corrected to shape (None, 1))
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),  # Input image B
            tf.TensorSpec(shape=(None, 1), dtype=tf.float32)             # Scalar features B (corrected to shape (None, 1))
        ),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)                  # Labels
    )
).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(
    train_dataset,
    epochs=30,  # Run for 30 epochs
    steps_per_epoch=100,  # Adjust this based on the size of your dataset
    verbose=1  # Print training progress
)

Epoch 1/500
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - classifier_accuracy: 0.4988 - loss: 1.6664
Epoch 1: val_loss improved from inf to 0.69556, saving model to /content/drive/MyDrive/Contrastive Learning/best_contrastive_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 336s 2s/step - classifier_accuracy: 0.4990 - loss: 1.6635 - val_classifier_accuracy: 0.4719 - val_loss: 0.6956 - learning_rate: 0.0010
Epoch 2/500
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - classifier_accuracy: 0.5246 - loss: 0.9518
Epoch 2: val_loss improved from 0.69556 to 0.68531, saving model to /content/drive/MyDrive/Contrastive Learning/best_contrastive_model.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 233s 2s/step - classifier_accuracy: 0.5246 - loss: 0.9512 - val_classifier_accuracy: 0.5203 - val_loss: 0.6853 - learning_rate: 9.9999e-04
Epoch 3/500
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - classifier_accuracy: 0.5338 - loss: 0.7908
Epoch 3: val_loss improved from 0.68531 to 0.67775, saving model to /content/drive/MyDrive/Contrast

KeyboardInterrupt: 

In [ ]:
base_network.save('/content/drive/MyDrive/Contrastive Learning/base_embedding_model.keras')

In [ ]:
base_network.save_weights('/content/drive/MyDrive/Contrastive Learning/model_weights.weights.h5')

# Triplet Model

In [2]:
def load_image(pothole_id, directory=image_dir, target_size=(224, 224)):
    img_path = os.path.join(directory, f'{pothole_id}.jpg')
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize the image
    return img

def standardize_features(labels_df):
    scaler = StandardScaler()
    labels_df[['mm_to_pixel_ratio']] = scaler.fit_transform(
        labels_df[['mm_to_pixel_ratio']]
    )
    return labels_df, scaler

def triplet_generator_hard(labels_df, batch_size=16):
    while True:
        anchors = []
        positives = []
        negatives = []
        scalar_anchor = []
        scalar_pos = []
        scalar_neg = []

        for _ in range(batch_size):
            anchor = labels_df.sample(1).iloc[0]
            positive = labels_df[labels_df['class'] == anchor['class']].sample(1).iloc[0]
            # Hard negative mining: choose the most similar class as the negative
            negative_class = labels_df[labels_df['class'] != anchor['class']].sample(1)['class'].iloc[0]
            negative = labels_df[labels_df['class'] == negative_class].sample(1).iloc[0]

            anchors.append(load_image(anchor['pothole_id']))
            positives.append(load_image(positive['pothole_id']))
            negatives.append(load_image(negative['pothole_id']))

            scalar_anchor.append([anchor['mm_to_pixel_ratio']])
            scalar_pos.append([positive['mm_to_pixel_ratio']])
            scalar_neg.append([negative['mm_to_pixel_ratio']])

        yield (
            {
                'input_anchor_image': np.array(anchors),
                'input_pos_image': np.array(positives),
                'input_neg_image': np.array(negatives),
                'input_scalar_anchor': np.array(scalar_anchor),
                'input_scalar_pos': np.array(scalar_pos),
                'input_scalar_neg': np.array(scalar_neg)
            },
            np.zeros((batch_size, 3))  # Dummy target for the triplet loss
        )

def triplet_dataset(labels_df, batch_size=16):
    return tf.data.Dataset.from_generator(
        lambda: triplet_generator_hard(labels_df, batch_size),
        output_signature=(
            {
                'input_anchor_image': tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                'input_pos_image': tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                'input_neg_image': tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                'input_scalar_anchor': tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
                'input_scalar_pos': tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
                'input_scalar_neg': tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
            },
            tf.TensorSpec(shape=(None, 3), dtype=tf.float32)  # Dummy target
        )
    )

def make_simplified_embedding_model(input_shape=(224, 224, 3), scalar_shape=(1,), embedding_dim=64, dropout_rate=0.2):
    inp = Input(shape=input_shape, name='input_image')

    # Fewer filters and layers
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same', name='conv_layer_1')(inp)
    c1 = MaxPooling2D((2, 2), name='pool_layer_1')(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_layer_2')(c1)
    c2 = MaxPooling2D((2, 2), name='pool_layer_2')(c2)

    c3 = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_layer_3')(c2)
    c3 = MaxPooling2D((2, 2), name='pool_layer_3')(c3)

    f1 = GlobalAveragePooling2D(name='global_avg_pool')(c3)

    scalar_input = Input(shape=scalar_shape, name='input_scalar')
    scalar_dense = Dense(32, activation='relu')(scalar_input)
    scalar_dense = Dropout(dropout_rate)(scalar_dense)

    combined = Concatenate(name='concat_image_scalar')([f1, scalar_dense])

    embedding = Dense(embedding_dim, name='embedding_layer')(combined)
    embedding = BatchNormalization(name='embedding_batch_norm')(embedding)

    return Model(inputs=[inp, scalar_input], outputs=embedding, name='simplified_embedding_model')

# Use the simplified embedding model in the triplet model
embedding_model = make_simplified_embedding_model()

def triplet_loss(margin=1.0):
    def _triplet_loss(y_true, y_pred):
        anchor = y_pred[:, 0, :]
        positive = y_pred[:, 1, :]
        negative = y_pred[:, 2, :]

        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)

        loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
        return loss
    return _triplet_loss

# Inputs
input_anchor_image = Input(shape=(224, 224, 3), name='input_anchor_image')
input_pos_image = Input(shape=(224, 224, 3), name='input_pos_image')
input_neg_image = Input(shape=(224, 224, 3), name='input_neg_image')

input_scalar_anchor = Input(shape=(1,), name='input_scalar_anchor')
input_scalar_pos = Input(shape=(1,), name='input_scalar_pos')
input_scalar_neg = Input(shape=(1,), name='input_scalar_neg')

# Generate embeddings
embedding_anchor = embedding_model([input_anchor_image, input_scalar_anchor])
embedding_positive = embedding_model([input_pos_image, input_scalar_pos])
embedding_negative = embedding_model([input_neg_image, input_scalar_neg])

# Concatenate embeddings into one tensor along a new axis using Lambda layer
merged_embeddings = Lambda(lambda x: tf.stack(x, axis=1))([embedding_anchor, embedding_positive, embedding_negative])

# Model to output merged embeddings for triplets
triplet_model = Model(
    inputs=[input_anchor_image, input_pos_image, input_neg_image, input_scalar_anchor, input_scalar_pos, input_scalar_neg],
    outputs=merged_embeddings
)

# Compile the model
triplet_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=triplet_loss(margin=1.0))

# Train the model for 30 epochs
history = triplet_model.fit(
    triplet_dataset(labels_df, batch_size=16),
    steps_per_epoch=100,
    epochs=30,
    verbose=1
)

# Save the embedding model after training
embedding_model.save('/content/drive/MyDrive/Contrastive Learning/triplet_embedding_model.keras')

Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 348s 3s/step - loss: 35.6173
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 48s 480ms/step - loss: 33.7711
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 45s 454ms/step - loss: 32.1816
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 45s 452ms/step - loss: 32.1564
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 459ms/step - loss: 29.2715
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 45s 451ms/step - loss: 26.8762
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 456ms/step - loss: 25.8626
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 44s 443ms/step - loss: 26.7316
Epoch 9/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 45s 452ms/step - loss: 23.1967
Epoch 10/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 455ms/step - loss: 24.7675
Epoch 11/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 44s 443ms/step - loss: 23.4034
Epoch 12/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 45s 449ms/step - loss: 21.1807
Epoch 13/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 46s 459ms/step - loss: 24.9610
Epoch 14/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 47s 467ms/step - loss: 21.5365
Epo